# Exploring the effect of grid options

`psiresp.GridOptions` controls the generation of the van der Waals' grid for the electrostatic potential computation. Users are not expected to interact directly with GridOptions very often, or to alter the default settings. However, new methods may do so, such as Resp2. I visualize the effect of changing the scale factors, density, and setting `grid_rmin` and `grid_rmax`.

In [1]:
import numpy as np
import psiresp
from psiresp.mixins.grid import GridOptions
from psiresp.tests.datafiles import ETHANOL_C1
import MDAnalysis as mda
import nglview as nv

In [2]:
u = mda.Universe(ETHANOL_C1)
nv.show_mdanalysis(u)

NGLWidget()

## Scale factors

Scale factors control how far from each atom the grid begins. For example, the default value constructs four shells, from `[1.4, 1.6, 1.8, 2.0]` times the atomic radius. Below I construct and visualize the first shell.

In [3]:
grid_options_shell_1 = GridOptions(vdw_scale_factors=[1.4])
shell_1 = grid_options_shell_1.generate_vdw_grid(u.atoms.elements, u.atoms.positions)

In [4]:
def add_points_to_view(view, points, color):
    points = list(np.concatenate(points))
    radii = [0.1] * len(points)
    color = color * (int(len(points) / 3))
    view.shape.add_buffer("sphere", position=points,
                          radius=radii, color=color)
    return view

view = nv.show_mdanalysis(u)
add_points_to_view(view, shell_1, [0, 0, 1])
view

NGLWidget()

The larger the scale factor, the further away from the molecule. Below I visualize the scale factor of 1.6 in green, and 1.8 and 2.0 in yellow.

In [5]:
grid_options_shell_2 = GridOptions(vdw_scale_factors=[1.6])
shell_2 = grid_options_shell_2.generate_vdw_grid(u.atoms.elements, u.atoms.positions)
grid_options_shell_3_4 = GridOptions(vdw_scale_factors=[1.8, 2.0])
shell_3_4 = grid_options_shell_3_4.generate_vdw_grid(u.atoms.elements, u.atoms.positions)

view = nv.show_mdanalysis(u)
add_points_to_view(view, shell_1, [0, 0, 1])
add_points_to_view(view, shell_2, [0, 1, 0])
add_points_to_view(view, shell_3_4, [1, 1, 0])
view

NGLWidget()

## Point density

The Resp2 classes have a higher point density than normal Resp, at 2.5. In the **Scale factors** section an example of a shell at 1.0 density was already given. Below we visualize the same shell at 2.5 density.

In [6]:
grid_shell_1_dense = GridOptions(vdw_scale_factors=[1.4],
                                 vdw_point_density=2.5)
shell_1_dense = grid_shell_1_dense.generate_vdw_grid(u.atoms.elements,
                                                     u.atoms.positions)
view = nv.show_mdanalysis(u)
add_points_to_view(view, shell_1_dense, [0, 0, 1])
view

NGLWidget()

## grid_rmin and grid_rmax

``grid_rmin`` and ``grid_rmax`` can be used to bound the grid generated, such that any point must fall between ``grid_rmin`` and ``grid_rmax`` of *any point*. I don't typically use this.

``use_radii`` controls which radius set to use. When originally implemented, Resp used the `"msk"` (Merz-Singh-Kollman) radii; however, some newer methods use Bondi (`"bondi"`) or Mantina (`"mantina"`) radii. Resp2 uses Bondi radii.

In [7]:
grid_shell_1_bounded = GridOptions(vdw_scale_factors=[1.4, 1.6, 1.8, 2.0],
                                   vdw_point_density=2.5, use_radii="bondi",
                                   grid_rmin=1.1, grid_rmax=2.3,)
shell_1_bounded = grid_shell_1_bounded.generate_vdw_grid(u.atoms.elements,
                                                     u.atoms.positions)
view = nv.show_mdanalysis(u)
add_points_to_view(view, shell_1_bounded, [0, 0, 1])
view

NGLWidget()